# Movie Reviews and Bag-of-Words Modeling

🎯 The goal of this challenge is to play with the ***Bag-of-words*** modeling of texts.

✍️ In the following dataset, we have $2000$ reviews classified either as _"positive"_ or _"negative"_.

In [1]:
import pandas as pd

data = pd.read_csv("reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [2]:
data.shape

(2000, 2)

## 1. Preprocessing

❓ **Question (Cleaning Text)** ❓

- Write a function `preprocessing` that will clean a sentence and apply it to all our reviews. It should:
    - remove whitespaces
    - lowercase characers
    - remove numbers
    - remove punctuation
    - tokenize
    - lemmatize
- You can store the cleaned reviews into a column called `cleaned_reviews`.
- Do not remove stopwords in this challenge, we will explain why in the section `3. N-gram modelling`

In [7]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence_cleaned = word_tokenize(sentence) ## tokenize 
   
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in tokenized_sentence_cleaned
    ]
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "s") 
        for word in tokenized_sentence_cleaned
    ]
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "a") 
        for word in tokenized_sentence_cleaned
    ]
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n") 
        for word in tokenized_sentence_cleaned
    ]
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "r") 
        for word in tokenized_sentence_cleaned
    ]
    
    cleaned_reviews = ' '.join(word for word in lemmatized)
    
    return cleaned_reviews


In [9]:
data['cleaned_reviews'] = data.reviews.apply(preprocessing)


In [10]:
data

,target,reviews,cleaned_reviews
0,neg,"plot : two teen couples go to a church party ,...",plot two teen couples go to a church party dri...
1,neg,the happy bastard's quick movie review \ndamn ...,the happy bastards quick movie review damn tha...
2,neg,it is movies like these that make a jaded movi...,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first feature...
4,neg,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ps...
...,...,...,...
1995,pos,wow ! what a movie . \nit's everything a movie...,wow what a movie its everything a movie can be...
1996,pos,"richard gere can be a commanding actor , but h...",richard gere can be a commanding actor but hes...
1997,pos,"glory--starring matthew broderick , denzel was...",glorystarring matthew broderick denzel washing...
1998,pos,steven spielberg's second epic film on world w...,steven spielbergs second epic film on world wa...


In [12]:
data.target.unique()

array(['neg', 'pos'], dtype=object)

❓ **Question (LabelEncoding)**❓

LabelEncode your target and store it into a column called `"target_encoded"`

In [13]:
# YOUR CODE HERE
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='if_binary', sparse = False) 
ohe.fit(data[['target']])
data['target'] = ohe.transform(data[['target']])

In [14]:
# Quick check
data.head()

,target,reviews,cleaned_reviews
0,0.0,"plot : two teen couples go to a church party ,...",plot two teen couples go to a church party dri...
1,0.0,the happy bastard's quick movie review \ndamn ...,the happy bastards quick movie review damn tha...
2,0.0,it is movies like these that make a jaded movi...,it is movies like these that make a jaded movi...
3,0.0,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first feature...
4,0.0,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ps...


## 2. Bag-of-Words modeling

❓ **Question (NaiveBayes with unigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a Bag-of-Words representation of the texts.

In [19]:
# YOUR CODE HERE
import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score

# Feature/Target
X = data["cleaned_reviews"]
y = data["target"]

# Pipeline vectorizer + Naive Bayes
pipeline_naive_bayes = make_pipeline(TfidfVectorizer(), 
                                     MultinomialNB())

# Cross-validation
cv_results = cross_validate(pipeline_naive_bayes, X, y, cv = 5, scoring = ["accuracy"])
average_accuracy = cv_results["test_accuracy"].mean()
np.round(average_accuracy,2)

0.82

## 3. N-gram modeling

👀 Remember that we asked you not to remove stopwords. Why? 

👉 We will train the Naive Bayes model with bigrams. Hence, in sentence like "I do not like coriander", it is important to scan the bigram "do not" to detect negativity in this sentence for example.

❓ **Question (NaiveBayes with bigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a 2-gram Bag-of-Words representation of the texts.

In [20]:
vectorizer = CountVectorizer(ngram_range = (2,2))
naivebayes = MultinomialNB()

X_bow = vectorizer.fit_transform(data.clean_reviews)

cv_nb = cross_validate(naivebayes,
                       X_bow,
                       data.target_encoded,
                       scoring = "accuracy")

round(cv_nb['test_score'].mean(),2)

AttributeError: 'DataFrame' object has no attribute 'clean_reviews'

🏁 Congratulations! Now, you know how to train a Naive Bayes model on vectorized texts.

💾 Don't forget to `git add/commit/push` your notebook...

🚀 ... and move on to the next challenge!